#### 同策略基金分析

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from function import *
from WindPy import w
w.start()

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [18]:
import pyecharts.options as opts
from pyecharts.charts import Line,Bar,Tab, Page
from pyecharts.components import Table

In [2]:
# 策略名称
strategy = "主观CTA"
# 辅助日期
year_begin_date = "2023-12-29"
month_begin_date = "2024-11-29"
week_begin_date = "2024-12-13"

In [37]:
basic_info = load_data("产品目录.xlsx")
basic_info

,strategy,fund_code,fund_name,benchmark_code,benchmark_name,threshold
0,主观CTA,SM6151,草本优益1号,NH0100.NHF,南华商品指数,-0.05
1,主观CTA,SM6151,草本致远1号,NH0300.NHF,南华农产品指数,-0.10
2,主观CTA,SGC034,济海实恒对冲二号,NH0100.NHF,南华商品指数,-0.10
3,主观CTA,ST6571,星恒哲萌1号,NH0100.NHF,南华商品指数,-0.05
4,量化CTA,SS0221,均成CTA1号,NH0100.NHF,南华商品指数,-0.10
5,量化CTA,SXF557,千衍九凌1号,NH0100.NHF,南华商品指数,-0.03
6,量化CTA,SCV067,众壹资产量合兴成一号,NH0100.NHF,南华商品指数,-0.05
7,量化CTA,SLS182,弘源多元化CTA,NH0100.NHF,南华商品指数,-0.05
8,量化CTA,SN3221,宏锡量化CTA7号,NH0100.NHF,南华商品指数,-0.05
9,量化CTA,SGA623,会世元丰CTA1号,NH0100.NHF,南华商品指数,-0.10


#### 表格

In [24]:
nav_dfs = None
nav_table = pd.DataFrame(
    columns=[
        "策略类型",
        "产品名称",
        "日期",
        "单位净值",
        "累计净值",
        "复权净值",
        "本年收益(%)",
        "当月收益(%)",
        "当周收益(%)",
    ]
)
for fund in basic_info["fund_name"]:
    # 导入净值数据
    files_list_series = pd.Series([i.stem for i in Path("./data").glob("*.xlsx")])
    nav_df_path = os.path.join("data",files_list_series[files_list_series.str.contains(fund)].item()+".xlsx")
    nav_df = load_data(nav_df_path)
    nav_df = get_standardized_data(nav_df)
    nav_df.rename(
        columns={
            "date": "日期",
            "nav_unit": "单位净值",
            "nav_accumulated": "累计净值",
            "nav_adjusted": "复权净值",
        },
        inplace=True,
    )
    # 计算年、月、周收益
    year_begin_nav = nav_df[nav_df["日期"] == year_begin_date]
    month_begin_nav = nav_df[nav_df["日期"] == month_begin_date]
    week_begin_nav = nav_df[nav_df["日期"] == week_begin_date]
    nav_head = nav_df.head(1)
    nav_tail = nav_df.tail(1)
    nav_tail["产品名称"] = fund
    nav_tail["策略类型"] = basic_info[basic_info["fund_name"] == fund]["strategy"].item()
    if not year_begin_nav.empty:
        year_begin_value = year_begin_nav["复权净值"].iloc[0]
        nav_tail["本年收益(%)"] = round(
            ((nav_tail["复权净值"].iloc[0] / year_begin_value) - 1) * 100, 2
        )
    else:  # 如果找不到2023-12-29净值，则使用第一条数据
        nav_tail["本年收益(%)"] = round(
            ((nav_tail["复权净值"].iloc[0] / nav_head["复权净值"].iloc[0]) - 1) * 100, 2
        )
    month_begin_value = month_begin_nav["复权净值"].iloc[0]
    nav_tail["当月收益(%)"] = round(
        ((nav_tail["复权净值"].iloc[0] / month_begin_value) - 1) * 100, 2
    )
    week_begin_value = week_begin_nav["复权净值"].iloc[0]
    nav_tail["当周收益(%)"] = round(
        ((nav_tail["复权净值"].iloc[0] / week_begin_value) - 1) * 100, 2
    )
    # 添加至表格
    nav_table = pd.concat([nav_table, nav_tail], ignore_index=True)
    custom_order = ["主观CTA", '量化CTA', "套利"]
    nav_table['策略类型'] = pd.Categorical(nav_table['策略类型'], categories=custom_order, ordered=True)
    nav_table.sort_values(by=['策略类型', '本年收益(%)'], inplace=True, ascending=[True, False], ignore_index=True)

In [ ]:
nav_table["日期"][0].

AttributeError: 'Timestamp' object has no attribute 'types'

In [25]:
table_index = Table()
headers = list(nav_table.columns)
rows = nav_table.values
table_index.add(headers, rows)
table_index.render_notebook()

策略类型,产品名称,日期,单位净值,累计净值,复权净值,本年收益(%),当月收益(%),当周收益(%)
主观CTA,草本致远1号,2024-12-20 00:00:00,1.037,3.01,4.3253,49.01,-1.89,-2.54
主观CTA,草本优益1号,2024-12-20 00:00:00,0.994,3.556,5.4755,16.72,-3.97,-1.39
主观CTA,星恒哲萌1号,2024-12-20 00:00:00,1.1133,1.5766,1.6899,12.04,0.36,0.17
主观CTA,济海实恒对冲二号,2024-12-20 00:00:00,1.052,1.948,2.2411,11.51,-0.28,-0.75
量化CTA,千衍九凌1号,2024-12-20 00:00:00,1.3131,1.7841,1.3131,31.31,2.53,0.53
量化CTA,量派CTA七号,2024-12-20 00:00:00,1.5975,1.5975,1.5975,31.15,0.72,-0.73
量化CTA,会世元丰CTA1号,2024-12-13 00:00:00,1.9887,1.9887,1.9887,17.96,1.5,0.0
量化CTA,弘源多元化CTA,2024-12-20 00:00:00,1.66,1.66,1.66,15.76,1.78,1.41
量化CTA,宏锡量化CTA7号,2024-12-20 00:00:00,1.2783,2.2005,2.8137,12.85,1.85,2.33
量化CTA,众壹资产量合兴成一号,2024-12-20 00:00:00,1.236,1.508,1.2612,11.05,0.33,1.64


#### 折线图

In [54]:
def get_data(strategy):
    strategy_info = basic_info.loc[(basic_info["strategy"] == strategy), :]
    strategy_info.reset_index(drop=True, inplace=True)
    nav_dfs = None
    for fund in strategy_info["fund_name"]:
        files_list_series = pd.Series([i.stem for i in Path("./data").glob("*.xlsx")])
        nav_df_path = os.path.join(
            "data",
            files_list_series[files_list_series.str.contains(fund)].item() + ".xlsx",
        )
        nav_df = load_data(nav_df_path)
        nav_df = get_standardized_data(nav_df)
        nav_df.rename(columns={"date": "日期", "nav_adjusted": fund}, inplace=True)
        nav_df = nav_df[["日期", fund]]
        if nav_dfs is None:
            nav_dfs = nav_df
        else:
            nav_dfs = pd.merge(nav_dfs, nav_df, on="日期", how="inner")
            nav_dfs = nav_dfs.set_index("日期")
            nav_dfs = nav_dfs.div(nav_dfs.iloc[0])
    return nav_dfs

In [55]:
get_line(get_data("主观CTA"), "主观CTA").render_notebook()

In [56]:
get_line(get_data("量化CTA"), "量化CTA").render_notebook()

In [57]:
tab = Tab()
tab.add(table_index, "各策略CTA产品涨跌幅")
tab.add(get_line(get_data("主观CTA"), "主观CTA"), "主观CTA走势")
tab.add(get_line(get_data("量化CTA"), "量化CTA"), "量化CTA走势")
tab.render("CTA策略收益跟踪.html")

'd:\\Desktop\\VScode\\Private_nav_research\\CTA策略收益跟踪.html'